# Purpose
Create historical measures data frames for gratitude projects


## Thanks
A dataframe where each row represents one each thanks 
+ Sender columns
    + Count of previously received thanks
    + Did they previously receive thanks? (binary indicator)
    + Count of previously sent thanks
    + Did they previously send thanks? (binary)
    +(summary information about the account)
    + When registered?
    + Number of previous edits
    + Had they previously edited the page associated with the thanks
    + Edited the page associated with the thanks within the next day after sending thanks
+ Receiver columns
    + Count of previously received thanks
    + Did they previously receive thanks? (binary indicator)
    + Count of previously sent thanks
    + Did they previously send thanks? (binary)
    + (summary information about the account)
    + When registered?
    + Number of previous edits
    + Over the next six months (could be three):
    + How many contributions they made
    + How many thanks they sent to others
+ Thanks-wide columns
    + What page it was
    + Thanks timestamp
    + How many previous edits did the page receive
    + When was the page first created?
    + (if easy, view count at the time of thanks)

## Love
+ A dataframe where each row represents one love action
+ Sender Columns
    + sender_prev_loves: How many loves did they previously send?
    + sender_prev_any.love: Did they previously send a love? (binary)
    + How many loves did they previously receive?
    + Did they previously receive a love (binary)
    + (summary information)
    + When registered?
    + Number of previous edits
+ Receiver Columns
    + receiver.prev.loves: How many loves did they previously send?
    + Did they previously send a love? (binary)
    + How many loves did they previously receive?
    + Did they previously receive a love (binary)
    + (summary information)
    + When registered?
    + Number of previous edits
    + Over the next six months
    + How many contributions they made
    + How many loves they sent to others
+ Love Columns
    + Timestamp
    + What kind of love (kitten, baklava)
    + Length of message
    + Count the number of links in the message
    + At the time of love-sending, how many pages had the sender and receiver both edited? (count)
    + At the time of love-sending, had the sender and receiver both edited at least one of the same pages (binary)


In [1]:
from sqlalchemy import create_engine
import sys, os
import pandas as pd
import numpy as np

In [2]:
constr = 'mysql+pymysql://{user}:{pwd}@{host}'.format(user=os.environ['MYSQL_USERNAME'],
                                                      pwd=os.environ['MYSQL_PASSWORD'],
                                                      host=os.environ['MYSQL_HOST'])
con = create_engine(constr)

#en_thanks = pd.read_sql("select * from enwiki_p.logging where log_type = 'thanks'", con)

In [4]:
!free

              total        used        free      shared  buff/cache   available
Mem:        8179272      555464     6814864       19392      808944     7342980
Swap:             0           0           0


In [ ]:
en_thanks.memory_usage().sum() / 10**6

In [84]:
df.head()

,log_id,log_type,log_action,log_timestamp,log_user,log_namespace,log_title,log_comment_id,log_comment,log_params,log_deleted,log_user_text,log_page
0,19254177,b'thanks',b'thank',b'20130820194639',89912.0,2,b'Blackfish',0.0,b'',b'a:0:{}',0,b'Matma Rex',2419281
1,19254178,b'thanks',b'thank',b'20130820194718',468168.0,2,b'Alan_ffm',0.0,b'',b'a:0:{}',0,b'Drzewianin',2010107
2,19254215,b'thanks',b'thank',b'20130820195307',432759.0,2,b'Basshuntersw',0.0,b'',b'a:0:{}',0,b'Stanko',1902059
3,19254221,b'thanks',b'thank',b'20130820195321',432759.0,2,b'Sir_Lothar',0.0,b'',b'a:0:{}',0,b'Stanko',425996
4,19254224,b'thanks',b'thank',b'20130820195353',304960.0,2,b'Wpisywacz',0.0,b'',b'a:0:{}',0,b'Bartholomaeus',3009335


### note on thank sizes
#### june 2018
+ enwiki has 1.5m thanks ~130megs
+ plwiki has 81k thanks ~2.5meg

In [57]:
r= con.execute("select count(*) from enwiki_p.logging where log_type = 'thanks'")

In [58]:
r.fetchall()

[(1477853,)]

In [66]:
all_thanks_sql = "select log_timestamp, log_title, log_user_text, log_page from plwiki_p.logging where log_type = 'thanks'"
thank_df = pd.read_sql(all_thanks_sql, con)

In [68]:
thank_df = thank_df.rename(mapper={'log_title':'receiver', 'log_user_text':'sender', 'log_timestamp':'timestamp'}, axis='columns')

In [1]:
thank_df.head()

NameError: name 'thank_df' is not defined

In [89]:
blackfish_df = thank_df[thank_df['receiver']==b'Blackfish']

In [102]:
def prev_thanks(user, send_or_receive, prior_to, df):
    time_cond = (df['timestamp'] < prior_to)
    user_cond = (df[send_or_receive] == user)
    prev_df = df[time_cond & user_cond]
    return len(prev_df)

In [110]:
thank_df.head()

,timestamp,receiver,sender,log_page
0,b'20130820194639',b'Blackfish',b'Matma Rex',2419281
1,b'20130820194718',b'Alan_ffm',b'Drzewianin',2010107
2,b'20130820195307',b'Basshuntersw',b'Stanko',1902059
3,b'20130820195321',b'Sir_Lothar',b'Stanko',425996
4,b'20130820195353',b'Wpisywacz',b'Bartholomaeus',3009335


In [120]:
thank_df1k = thank_df[:1000]

In [121]:
len(thank_df1k)

1000

In [126]:
grps = thank_df.groupby(by='receiver')

In [123]:
thank_df['receiver_prev_received'] = thank_df.apply(lambda row: prev_thanks(user=row[1], send_or_receive='receiver', prior_to=row[0], df=thank_df), axis=1)

In [128]:
thank_df['sender_prev_received'] = thank_df.apply(lambda row: prev_thanks(user=row[2], send_or_receive='receiver', prior_to=row[0], df=thank_df), axis=1)

In [129]:
thank_df['sender_prev_sent'] = thank_df.apply(lambda row: prev_thanks(user=row[2], send_or_receive='sender', prior_to=row[0], df=thank_df), axis=1)

In [130]:
thank_df['receiver_prev_sent'] = thank_df.apply(lambda row: prev_thanks(user=row[1], send_or_receive='sender', prior_to=row[0], df=thank_df), axis=1)

In [132]:
thank_df.to_pickle('thank_pl_v1.pickle')

In [134]:
!du -h thank_pl_v1.pickle

6.4M	thank_pl_v1.pickle


## Checkpoint

In [27]:
thank_df = pd.read_pickle('thank_pl_v1.pickle')

In [58]:
def natural_integer_test(seq):
    '''tests if a series is a natural integer sequence using a property'''
    l = len(seq) - 1 #minus 1 because we always start with a zero
    sum_identity = (l*(l+1))/2
    assert sum_identity == seq.sum()
    
def monotonic(x):
    assert np.all(np.diff(x) >= 0)    

In [61]:
blackfish_r = thank_df[thank_df['receiver']==b'Blackfish']
blackfish_s = thank_df[thank_df['sender']==b'Blackfish']
blackfish_rpr = blackfish_r['receiver_prev_reiceived']
blackfish_rps = blackfish_r['receiver_prev_sent']
blackfish_sps = blackfish_s['sender_prev_sent']
blackfish_spr = blackfish_s['sender_prev_reiceived']

In [65]:
natural_integer_test(blackfish_rpr)
natural_integer_test(blackfish_sps)
monotonic(blackfish_spr)
monotonic(blackfish_rps)

## Edit histories

In [ ]:
all_user_names_that_thanked = '''use plwiki_p;
  select user_id, user_name from user uu
	join (select distinct uname from (
 			select distinct log_title as uname from logging where log_type = 'thanks'
    		union
     		select distinct log_user_text as uname from logging where log_type = 'thanks') k
			) l
  on uu.user_name = l.uname'''

In [3]:
thank_revisions = '''
select thank_users.user_name, rev_timestamp from plwiki_p.revision revs
inner join  (select user_id, user_name from plwiki_p.user userids
	join (select distinct uname from (
 			select distinct log_title as uname from plwiki_p.logging where log_type = 'thanks'
    		union
     		select distinct log_user_text as uname from plwiki_p.logging where log_type = 'thanks') send_and_rec
			 ) thank_mentioned
  on userids.user_name = thank_mentioned.uname) thank_users
  on revs.rev_user = thank_users.user_id 

'''

In [ ]:
rev_df = pd.read_sql(thank_revisions, con)

In [ ]:
rev_df.to_pickle('revs_pl_v1.pickle')

In [12]:
len(rev_df)

23616495

## alternative strategy for revisions
create a dict or dataframes, one per user using
```
use plwiki_p;
select rev_user, rev_timestamp from revision r
inner join (
  select user_id, user_name from user where user_name = ''
) u on r.rev_user = u.user_id 

```

In [ ]:
rev_sql =
'''
use plwiki_p;
select rev_user, rev_timestamp from revision r
inner join (select user_id, user_name from user where user_name = 'Blackfish') u on r.rev_user = u.user_id 
'''

### Question
+ is log_title thanking log_user_text, or vice-versa?
### Answer
+ it is  log_title **was thanked by** 	log_user_text

In [51]:
edf = pd.read_sql("select log_timestamp, log_user, log_title, log_user_text, log_page from enwiki_p.logging where log_type = 'thanks' and log_title = 'Maximilianklein' ", con)

In [53]:
eedf = pd.read_sql("select log_timestamp, log_user, log_title, log_user_text, log_page from enwiki_p.logging where log_type = 'thanks' and log_user_text = 'Maximilianklein' ", con)

In [46]:
df['log_comment_id'].value_counts()

0.0    76857
5.0     4341
Name: log_comment_id, dtype: int64

In [47]:
df['log_comment']

0        b''
1        b''
2        b''
3        b''
4        b''
5        b''
6        b''
7        b''
8        b''
9        b''
10       b''
11       b''
12       b''
13       b''
14       b''
15       b''
16       b''
17       b''
18       b''
19       b''
20       b''
21       b''
22       b''
23       b''
24       b''
25       b''
26       b''
27       b''
28       b''
29       b''
        ... 
81168    b''
81169    b''
81170    b''
81171    b''
81172    b''
81173    b''
81174    b''
81175    b''
81176    b''
81177    b''
81178    b''
81179    b''
81180    b''
81181    b''
81182    b''
81183    b''
81184    b''
81185    b''
81186    b''
81187    b''
81188    b''
81189    b''
81190    b''
81191    b''
81192    b''
81193    b''
81194    b''
81195    b''
81196    b''
81197    b''
Name: log_comment, Length: 81198, dtype: object

### namespaces always seem to be = 2